# An Orderbook Timeline Displayed in Jupyter Notebook
- October 2025, Alan Lenarcic

Note to really run this notebook you need a working Python environment with ```pyrus_kaiko`` package installed and the ```gpuorderbook``` functionality compiled with ES builder.

See README for ```gpuorderbook``` installation.

We hope to be able to compile this Notebook to HTML using nbconvert

## Here we use anywidget to render javascript inside a jupyter notebook.  We use ```pyrus_kaiko`` package to simulate orderbook data.

- nbconvert will be required to export as html

In [1]:
from pyrus_kaiko import pyrus_kaiko
## Note that pyo3 has changeable module inclusion adivce
## Here we use a "from pyrus_package import pyrus_package" interface which runs our "__init__.py" script

 pyrus_kaiko first try to from import
"asjoin_py" is part of "dir(pyrus_kaiko)"
------------------- pyrus_kaiko::__init__.py  running ---------- ---
-- dir(pyrus_kaiko) is [__all__,__doc__,__file__,__loader__,__name__,__package__,__spec__,asjoin_py,kaiko_fob,kaiko_make_u_fob,pyrus_verify_prices,test_pa,unsorted_asjoin_py]
--- Thank you for running init.
-- dir(pk) is [__all__,__doc__,__file__,__loader__,__name__,__package__,__spec__,asjoin_py,kaiko_fob,kaiko_make_u_fob,pyrus_verify_prices,test_pa,unsorted_asjoin_py]
-------------------------------------------------------------


In [2]:
import os; 
from pyrus_kaiko import pyrus_kaiko;
from pyrus_kaiko import ob;

nGen=20000; #nGen = 5000; 
nMidPoints = 4000;
import numpy as np; import pandas as pd;
import pyarrow as pa;
out_t, nr, v_d_fake, bprices, sprices, \
  write_sum_q, write_sum_pq, write_avp, write_worstpi, \
  write_nlevels, write_atq, \
  v_w, verbose, time_minmax, a_Fake_OB, MidTable = \
  ob.fake_for_algo(nGen = nGen,nMidPoints=nMidPoints,AddMidPrice=True, deltaSec=.01, v_bhit=False,fast_check="PARALLEL");



 pyrus_marketbook first try to from import
------------------- pyrus_marketbook:__init__.py  running ---------- ---
-- dir(pyrus_makretbook) is [__all__,__doc__,__file__,__loader__,__name__,__package__,__spec__,asjoin_py,b2v_test,cumulate,dynarray,dynarray2,example_take,limit_hit,out_recarray,out_tmprsd,pyrus_bbo_algo,pyrus_verify_prices,raw_string_fun_example,sip_algo,sip_arrow,take_table,unsorted_asjoin_py]
--- Thank you for running init.
-- dir(pb) is [__all__,__doc__,__file__,__loader__,__name__,__package__,__spec__,asjoin_py,b2v_test,cumulate,dynarray,dynarray2,example_take,limit_hit,out_recarray,out_tmprsd,pyrus_bbo_algo,pyrus_verify_prices,raw_string_fun_example,sip_algo,sip_arrow,take_table,unsorted_asjoin_py]
-------------------------------------------------------------
Fake_OB(Vb=1,nG=20000):  -- Begin
Fake_OB(Vb=1,nG=20000):  -- Fake_OB Generated mid_of_v, now makign add points
Fake_OB(Vb=1,nG=20000): : testing v_t1lc length 30000 for data against midpoints at 6000 points.
F

In [11]:
# fake NBBO from the fake Midpoint
import copy
pd_dem = copy.copy(MidTable);
pd_dem = pd_dem[(pd_dem.time <= time_minmax[1]) & (pd_dem.time.shift(-1,fill_value=time_minmax[1])  >= time_minmax[0])].reset_index(inplace=False, drop=True)
pd_dem['best_bid'] = pd_dem['nbm'] - .01;
pd_dem['best_ask'] = pd_dem['nbm'] + .01;
pd_dem['b_sum_q_1'] = 100;
pd_dem['s_sum_q_1'] = 100;


In [12]:
"""
# Use if you want to calculate bestbid/ask from an algorithm
out_t['time'] = out_t['time'].astype('datetime64[ns]')
kalgoint_basic = np.array([0], dtype=np.uint8)[0];
v_d = np.asarray([1], dtype=np.int64); verbose_book = 0;
import time; 
time_00 = time.time();
out_PMB = pyrus_marketbook.pyrus_bbo_algo(out_t, nr, v_d, 
  bprices, sprices, kalgoint_basic,
  1, 0, 0, 0, 0, 0,
  v_w, verbose_book, time_minmax.astype(np.int64) );
time_01 = time.time();
print("pd_dem calculated from " + str(len(out_t)) + " recs in " + str(time_01-time_00) + " sec.");
pd_dem = pa.Table.from_batches([out_PMB]).to_pandas();
pd_dem = pd_dem[(pd_dem['best_bid'] > 0) & (pd_dem['best_ask'] > 0)].reset_index(inplace=False, drop=True)
"""

'\n# Use if you want to calculate bestbid/ask from an algorithm\nout_t[\'time\'] = out_t[\'time\'].astype(\'datetime64[ns]\')\nkalgoint_basic = np.array([0], dtype=np.uint8)[0];\nv_d = np.asarray([1], dtype=np.int64); verbose_book = 0;\nimport time; \ntime_00 = time.time();\nout_PMB = pyrus_marketbook.pyrus_bbo_algo(out_t, nr, v_d, \n  bprices, sprices, kalgoint_basic,\n  1, 0, 0, 0, 0, 0,\n  v_w, verbose_book, time_minmax.astype(np.int64) );\ntime_01 = time.time();\nprint("pd_dem calculated from " + str(len(out_t)) + " recs in " + str(time_01-time_00) + " sec.");\npd_dem = pa.Table.from_batches([out_PMB]).to_pandas();\npd_dem = pd_dem[(pd_dem[\'best_bid\'] > 0) & (pd_dem[\'best_ask\'] > 0)].reset_index(inplace=False, drop=True)\n'

In [13]:
nTrades = 200; trade_i = np.asarray(np.unique(np.random.choice(np.arange(len(pd_dem)), nTrades, replace=False)))
ts = np.random.choice(np.asarray([0,1]),nTrades)
trades =  pd_dem.iloc[trade_i].reset_index(inplace=False, drop=True);
tprice = trades['best_bid'].to_numpy();
tprice[ts == 0] == trades['best_ask'].to_numpy()[ts == 0]
tqty = trades['b_sum_q_1'];  tqty[ts==0] == trades['s_sum_q_1'][ts==0];
trades = pd.DataFrame({'time':trades['time'],'price':tprice,'qty': (-2*ts + 1) * tqty})
trades['time'] = (trades['time']-pd_dem['time'][0]).astype(np.int64)/1000000000.0

trades

,time,price,qty
0,0.10,10007.99,100
1,0.25,10016.99,100
2,0.56,10012.99,100
3,0.84,10001.99,100
4,1.18,9988.99,100
...,...,...,...
195,38.18,9974.99,100
196,38.19,9970.99,100
197,38.28,9969.99,-100
198,38.84,9974.99,100


In [18]:
dt = str(pd_dem['time'][0])[0:10]; st_time =  str(pd_dem['time'][0])[11:26]; unit = 0;
time0 = pd_dem['time'][0];
nbbo = pd_dem[['time','best_bid','best_ask']].reset_index(inplace=False, drop=True); nbbo.columns = ['time','nbb','nbo']; nbbo['time'] = ((nbbo['time'] - nbbo['time'][0]).astype(np.int64)) / 1000000000.0
tmin = np.min(nbbo['time']); tmax = np.max(nbbo['time']);
inner_pmin = np.min(nbbo['nbb']);  inner_pmax = np.max(nbbo['nbo']);
pmin = inner_pmin - 1 * (inner_pmax - inner_pmin);  pmax = inner_pmax + 1 * (inner_pmax - inner_pmin);
height = 400; width = 800;
lwd_h = .01; lwd_w = (400/800) * lwd_h;
max_qty = np.max(out_t['cm_qty']); pow_qty = .25;
trade_mul_fac=.1; msg_mul_fac = .1;
buys = a_Fake_OB[a_Fake_OB['bs'] == b'b'][['price','qty','t0','t1']].reset_index(inplace=False, drop=True); buys['t0'] = (buys['t0'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  buys['t1'] = (buys['t1'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
buys.columns = ['price', 'qty', 'open','close']
buys

sells = a_Fake_OB[a_Fake_OB['bs'] == b's'][['price','qty','t0','t1']].reset_index(inplace=False, drop=True); sells['t0'] = (sells['t0'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  sells['t1'] = (sells['t1'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
sells.columns = ['price', 'qty', 'open','close']
sells
data_type = 'generated_data';

In [19]:
out_t
mxtime = np.max(out_t['time']);
buys = out_t[out_t['side'] == b'b'][['price','cm_qty','time']].reset_index(inplace=False,drop=True);
close = buys['time'].shift(-1,fill_value=mxtime);
close[buys['price'] != buys['price'].shift(-1,fill_value=-1)] = mxtime; buys['close'] = close;  buys.columns = ['price','qty','open','close'];  
buys['open'] = (buys['open'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  
buys['close'] = (buys['close'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
buys = buys[buys.qty > 0].reset_index(inplace=False, drop=True)

sells = out_t[out_t['side'] == b's'][['price','cm_qty','time']].reset_index(inplace=False,drop=True);
close = sells['time'].shift(-1,fill_value=mxtime);
close[sells['price'] != sells['price'].shift(-1,fill_value=-1)] = mxtime; sells['close'] = close;  sells.columns = ['price','qty','open','close'];  
sells['open'] = (sells['open'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;  
sells['close'] = (sells['close'].astype('datetime64[ns]') - time0).astype(np.int64) / 1000000000.0;
sells = sells[sells.qty > 0].reset_index(inplace=False, drop=True);
sells

,price,qty,open,close
0,9831.0,614.0,25.964710,25.968813
1,9831.0,423.0,26.176786,26.186096
2,9831.0,933.0,26.217562,26.219443
3,9832.0,143.0,25.940938,25.949300
4,9834.0,859.0,25.552498,25.594344
...,...,...,...,...
17633,10174.0,489.0,5.502543,6.583184
17634,10176.0,723.0,36.374738,40.010000
17635,10194.0,968.0,7.104873,7.313836
17636,10204.0,440.0,7.341543,11.470834


In [20]:
test_data = { "dt": dt, 'st_time':st_time, 'unit':unit, 'tmin':tmin, 'tmax':tmax, 'pmin':pmin, 'pmax':pmax,'height':height, 'width':width,
                'lwd_h':lwd_h, 'lwd_w':lwd_w, 'max_qty':max_qty, 'pow_qty':pow_qty,'trade_mul_fac':trade_mul_fac, 'msg_mul_fac':msg_mul_fac,
                'nbbo':nbbo.to_dict(orient='list'),'buys':buys.to_dict(orient='list'),'sells':sells.to_dict(orient='list'),'trades':trades.to_dict(orient='list'),'data_type':'generated_data'};
import json;
aD = json.dumps(test_data)

In [21]:
import pathlib
import anywidget
import traitlets
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }</style>"))


from IPython.display import HTML, display
html_code = "<h1>Hello, world!</h1><p>This is a paragraph.</p>"
install_div_space = \
"""<div id="gl_div" name="gl_div"> <div>"""
display(HTML(install_div_space))

class obwidget(anywidget.AnyWidget):
    _esm = "./outwidget/static/widget_index.js"
    _css = "./outwidget/static/widget_index.css"
    data = traitlets.Dict(test_data).tag(sync=True)
    value = traitlets.Int(0).tag(sync=True)

obwidget()

obwidget(data={'dt': '2024-10-03', 'st_time': '10:30:31.990000', 'unit': 0, 'tmin': 0.0, 'tmax': 40.01, 'pmin'…

In [8]:
import pathlib
import anywidget
import traitlets
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% ! important; }</style>"))


from IPython.display import HTML, display
html_code = "<h1>Hello, world!</h1><p>This is a paragraph.</p>"
install_div_space = \
"""<div id="gl_div" name="gl_div"> <div>"""
display(HTML(install_div_space))

class obwidget(anywidget.AnyWidget):
    _esm = "./outwidget/static/widget_index.js"
    _css = "./outwidget/static/widget_index.css"
    value = traitlets.Int(0).tag(sync=True)

obwidget()

obwidget()